# "Online Store Visitors Behavioral Analysis"
> "Exploring 20M rows of online store data"

- toc: false
- branch: master
- badges: true
- hide_github_badge: true
- comments: true
- categories: [pandas, EDA]
- image: images/ecommerce-analytics.jpg
- use_plotly: true
- hide: false
- search_exclude: true
- metadata_key1: ecommerce_analysis
- metadata_key2: Pandas


A while ago, I found [this dataset](https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop) on **Kaggle** which contains 20M user events from an online store. Its size over two Gigabite and the owner challenged data scientists to dig and find insights. so I took this challenge.  
the dataset has been separated into five files, each file contains events for an entire month. Starting from Octobor 2019 to Febrary 2020.

>Note: If you are a technical girl/gey [click here](https://saaleh2.github.io/ALHODAIF-Portfolio/pandas/eda/2021/11/23/My-First-Project.html#Let-us-explore-our-data) to skip to the analysis.

# For non-technicals
Before starting this analysis I will explain some technical terms so they become easy to digest while reading this post.    

- __events record__    
First of all, what is **event**?&emsp;an **event** is a recod of user actions in apps and websites. Each click you do in apps and websites will be recorded for future analysis and other purposes.   
The following example is simplifying how events recorded in a databases:    

In [1]:
#hide_input
# example data

example = {'event time':['2019/1/1 - 8:00 PM', '2019/1/1 - 8:05 PM', '2019/1/2 - 1:00 AM', '2019/1/2 - 1:10 AM', '2019/1/3 - 11:00 AM'], 
           'event type':['add to cart', 'purchase', 'view item', 'add to cart', 'view item'], 
           'product id':['milk', 'milk', 'Iphone 13', 'Iphone 13', 'soap'], 
           'category id':['food', 'food', 'electronics', 'electronics', 'cleaners'],
           'brand':['Almarai', 'Almarai', 'Apple', 'Apple', 'NaN'], 
           'price':[10.50, 10.50, 1499, 1499, 15], 
           'user id':['saleh', 'saleh', 'salman', 'salman', 'saleh'], 
           'user session':[1115,1115,1125,1125, 2334]}
df = pd.DataFrame(example, index=[1,2,3,4,5])
df

,event time,event type,product id,category id,brand,price,user id,user session
1,2019/1/1 - 8:00 PM,add to cart,milk,food,Almarai,10.5,saleh,1115
2,2019/1/1 - 8:05 PM,purchase,milk,food,Almarai,10.5,saleh,1115
3,2019/1/2 - 1:00 AM,view item,Iphone 13,electronics,Apple,1499.0,salman,1125
4,2019/1/2 - 1:10 AM,add to cart,Iphone 13,electronics,Apple,1499.0,salman,1125
5,2019/1/3 - 11:00 AM,view item,soap,cleaners,NaN,15.0,saleh,2334


- **How to read the events data?**    
The `event time` column  records when did the event happen. And the `event type` column  specify what the user did. `product id`, `category id`, and `user id` columns are always represented in numbers, but here we used text instead to simplify it.  It's easier for computers to use numbers rather than texts especially when there are duplicates, like when two users have the same name.        
__Sessions__    
Before explaining `user session` column, let's clear out what a **session** means.    
A **session**    is a group of user interactions with the website or app that take place within 30 minutes. So let's say you opened Noon app at 8:00 PM and start viewing items until 8:29 PM. All events that have been recorded during this period will be given a single **session** reference number. And when you open Noon again the next day it will be considered as a new session and a new reference number will take a palce. Notice how the `user id` "saleh" got a new `user session` reference number after coming back on 2019/1/3.
  
### Empty values
If you look back at row number 5 you will see that `brand` cells are **_NaN_**. Nan is actually not a brand, it simply means empty cell. Sometimes you will see **_null_** instead but they are the same thing.
___

&nbsp;    

## Let us explore our data
We're going first to look at our data structure and prepare the data for analysis. 

>Tip: Non-technicals can skip codes in the black boxes and read outputs (results) of the code under it.

In [1]:
#collapse_show
# importing all necessary libraries for this analysis.
import pandas as pd
import numpy as np
import plotly.express as px

# Reading the data and taking a quick look at data structure.
oct_file = pd.read_csv(r'D:\saleh\kaggle\2019-Oct.csv')
nov_file = pd.read_csv(r'D:\saleh\kaggle\2019-Nov.csv')
dec_file = pd.read_csv(r'D:\saleh\kaggle\2019-Dec.csv')
jan_file = pd.read_csv(r'D:\saleh\kaggle\2020-Jan.csv')
feb_file = pd.read_csv(r'D:\saleh\kaggle\2020-Feb.csv')
dataset = pd.concat([oct_file, nov_file, dec_file, jan_file, feb_file], ignore_index=True)
dataset.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9


&nbsp;  
The dataset has 9 columns, each row contains users' actions like adding items to the cart or viewing an item. At a glance, we see that our dataset needs some manipulation and cleaning. The `brand` column has empty values, but we will check in a minute how many empty cells are out there.   
Now let's see columns data type. It's important to make sure that data are given the right type so our tools don't mess up when dealing with data.

In [2]:
#collapse_show
# Calling dataset info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.4+ GB


Here, the `event_time` column is been signed as `object` type, which will make our life harder when we do time-based analysis. So we will convert it to `DateTime` data type:

In [3]:
#collapse_show
# changing event_time column from object to DateTime column
dataset['event_time'] = pd.to_datetime(dataset['event_time'])

&nbsp;  
Next we will check empty values.

In [4]:
#collapse_show
print('_______empty values_______\n', dataset.isna().sum())

_______empty values_______
 event_time              0
event_type              0
product_id              0
category_id             0
category_code    20339246
brand             8757117
price                   0
user_id                 0
user_session         4598
dtype: int64


&nbsp;  
The `category_code` and `brand` columns are 98% and 43% empty respectively, so we will get rid of them to make my cute machine fly faster.  
&nbsp;

In [5]:
#collapse_show
# dropping "category_code" and "brand" column
dataset.drop(columns=['category_code', 'brand'], inplace=True)

&nbsp;  
Lastly, let's check if there are any outliers so that we don't fall into that old mistake. **"garbage in -> garbage out"**
&nbsp;  
&nbsp;

In [6]:
#collapse_show
# counting number of events per user
dataset.groupby('user_id', as_index=False).agg(number_of_events=(
    'event_type', 'count')).nlargest(10, 'number_of_events').reset_index()

,index,user_id,number_of_events
0,281608,527021202,26752
1,494091,557616099,9903
2,985991,583884978,9420
3,81834,419558969,8283
4,165553,476450673,8122
5,147504,467810091,8066
6,505313,557956487,7735
7,377367,550388516,7556
8,391614,552908674,7419
9,229928,506877330,6909


&nbsp;  
The first user have an enormous number of events which could mean that it's not human or one of the employees doing their work using a user account. Possibilities are endless so we will exclude him anyway.
&nbsp;    
&nbsp; 

In [7]:
#collapse_show
# users list
outliers = [527021202]
# excluding outliers
dataset.drop(dataset[dataset['user_id'].isin(outliers)].index, inplace=True)

___
&nbsp; 
## What are we looking for?
__<mark>Good analysis comes out from good questions.</mark>__ For any online store, the most important thing is __conversion rate__. which basically means: 
out of all visitors to the website, how many of them actually placed orders.  
Therefore, we will build our analysis around that and try to find insights that can help the online store to understand their visitors' behavior and make better campaigns.


### What is the current conversion rate?

__"If you can't measure it you can't manage it".__&nbsp; Knowing the current state will help the stakeholders to measure whether if they are making any difference after using these analysis insights or not.

In [8]:
#collapse_show
# subsetting purchase events
# I found purchases with negative values, mostly it's returned orders.
purchases = dataset.query('event_type == "purchase" & price > 0')

# counting the total number of orders
orders_count = len(purchases['user_session'].unique())

# counting number of visitors
visitors_count = len(dataset['user_id'].unique())

# calculating conversion rate (number of orders / number of visitors)
conv_rate = round((orders_count / visitors_count),2)
print(f'Current conversion rate is:\033[1m %{conv_rate}')

Current conversion rate is: %0.09


Which is below any industry average. For more on average conversion rate by industry check out [this](https://www.invespcro.com/blog/the-average-website-conversion-rate-by-industry/) informative post.
___
&nbsp;    

## When do users visit the store?
Finding when users visit the store is crucial for digital marketing teams in many ways. The team can adjust their campaigns during these times which will lead to better targeting new customers and reduce the cost of their campaigns.  
In general, we want to know when users visit the store, and whether their behavior changes during weekends or not.

### Most active day of the week

In [14]:
#collapse_show
# Create 'date' column
dataset['date'] = dataset['event_time'].dt.date
# unique users in each day
unieq_users = dataset.drop_duplicates(subset=['user_id', 'date']).copy()
# Create 'dayname' column to group by it
unieq_users['dayname'] = unieq_users['event_time'].dt.day_name()
#aggregating data
count_vistors = unieq_users['dayname'].value_counts()
#visualizing data
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
fig = px.bar(count_vistors, x=count_vistors.index, y=count_vistors.values, template='plotly_white')
fig.update_layout(xaxis={'title':'', 'categoryorder':'array', 'categoryarray':days_order}, 
                      yaxis={'title':'Number of visitors'}, bargap=0.4)
fig.show()

Suprisengly, the most active days are the weekdays, not the weekends.

### What are the peak times?
Let's be more specific and find out when do users visit the store during the day. we're going to split weekdays from the weekends since weekends tend to have different routines and potentially dissimilar behavior. 

In [10]:
#collapse_show
# Creating hour and 'day of week' column
dataset['Hour'] = dataset['event_time'].dt.hour
dataset['d_of_week'] = dataset['event_time'].dt.weekday
# tagging each event as 'Weekend' or 'Weekday' time
wends_wdays = []

for ind, row in dataset.iterrows():
    if row['d_of_week'] < 5:
        wends_wdays.append('Weekday')
    else:
        wends_wdays.append('Weekend')

dataset['Wend/Wday'] = wends_wdays


Now we visualize peak times.

In [20]:
#collapse_show
# count the number of users per hour
peak_times = dataset.groupby(['Hour', 'Wend/Wday'], as_index=False).agg(
    {'user_id': pd.Series.nunique}).rename(columns={'user_id': 'Number of visitors'})

# plotting data
fig2 = px.line(peak_times, x='Hour', y='Number of visitors', color='Wend/Wday', 
               markers=True, symbol='Wend/Wday', template='plotly_white')
fig2.show()

We have slightly different behavior between weekends and weekdays, where weekdays have two peaks during the day, but both lines drop dramatically after 8 pm.
___

## Top 10 products
One of the ways to raise the conversion rate is to expand the range of the products. A key insight to that is to find top selling products and find expandable ones. Now we will detect the top 10 products and their categories.

In [12]:
#collapse_show
# count top 10 products
top_products = purchases.groupby('product_id', as_index=False)[
    'event_type'].count().nlargest(10, 'event_type')
# subsetting 'product_id' & 'category_id' columns
prod_cat = purchases[[
    'product_id', 'category_id']].drop_duplicates('product_id')
# selecting top 10 products with their category
top_10 = prod_cat[prod_cat['product_id'].isin(
    top_products['product_id'])].reset_index(drop=True)
top_10.index += 1
top_10

,product_id,category_id
1,5700037,1487580009286598681
2,5751383,1487580005092295511
3,5751422,1487580005268456287
4,5815662,1487580006317032337
5,5849033,1487580005092295511
6,5809912,1602943681873052386
7,5854897,1487580009445982239
8,5802432,1487580009286598681
9,5304,1487580009471148064
10,5809910,1602943681873052386


Out of the top 10 products, eight start with **148758** `category_id`, which represents the key number of the main category (ex. Food), and then the subcategory number comes after(ex. Rice).  
___

## Average session duration
Another important KPI in ecommerce is the **average session duration**. a session is a group of user interactions with the website that take place within 30 minutes. sadly, the sessions of this store were not registered in the right way. many sessions lasted 151 days and others have 0-second duration as shown below. so we can not use this data in our analysis.

In [13]:
#collapse_show
# extracting start time and end time of each session
ses_st_end = dataset.groupby('user_session', as_index=False).agg(
    min_=('event_time', np.min), max_=('event_time', np.max))
# subtract end from start to get session duration
ses_st_end['session_duration'] = ses_st_end['max_'] - ses_st_end['min_']
# select only needed columns
ses_duration = ses_st_end.loc[:, ['user_session', 'session_duration']].sort_values(
    'session_duration', ascending=False).reset_index(drop=True)
ses_duration

,user_session,session_duration
0,ae74cec4-ae31-4470-8484-84c3a75365d3,151 days 14:45:56
1,beac319a-88e8-43db-98e9-d6cd6184f444,151 days 11:10:49
2,099fefe4-a74c-4dae-b9c2-fe15dea34ff1,151 days 10:39:38
3,5b9bcf07-5c80-4f98-84dd-cad0883e0477,151 days 09:40:47
4,285e8547-29b3-49d2-b503-5ca9a60413cc,151 days 05:47:39
...,...,...
4518533,738a61cc-925e-48c8-ba6b-0ad258104504,0 days 00:00:00
4518534,738a6219-3a52-4915-a54b-2a75d3624a88,0 days 00:00:00
4518535,738a63f4-da1c-4f7b-8492-e2269d4f7c43,0 days 00:00:00
4518536,738a6949-bd26-4f23-8b26-7272f36eb3da,0 days 00:00:00


___
## Conclusions
The conversion rate tells us a lot about the store's performance. With a few little changes, the store can raise it without any additional costs. One of the ways is to create urgency feeling by using quantity countdown so the user act fast. Also, adjusting campaigns during peak times on Sunday, Monday, and Tuesday will bring more visits to the store, either new visitors or retargeting current users. The store can focus on top-selling categories and reduce others while testing new categories in small quantities.  
Data quality has some issues and the store data need to do some enhancements to have high-quality data in order to make better decisions in the future and a better understanding of visitors' behavior.